In [2]:
import numpy as np
from pydub import AudioSegment
import audiosegment
import cv2
import pdb
import os

import pickle as pkl

import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
%load_ext autoreload
%autoreload 2
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# directory where all my music lives
MUSIC_DIR = '/Users/timibennatan/Music/iTunes/iTunes Media/Music/**'
SNIPPETS_PER_SONG = 30 # extract 30 one second snippet from each song

# # create a generator of audiosegments for each song
songs = read_songs(MUSIC_DIR)
songs

Found: 3495 Files


<generator object read_songs.<locals>.<genexpr> at 0x11c05ff68>

In [28]:
def save_npy(arr,path):
    with(open(path, 'wb')) as handle:
        np.save(handle,arr)

In [20]:
pooler = get_pooler()

/Users/timibennatan/Desktop/Neuro-Soundtrack/feature_extraction/utils.py:96: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ma...)`
  model = Model(inputs = input_img, output = pool)


In [33]:
# create a generator of audiosegments for each song
songs = read_songs(MUSIC_DIR)

def save_data(printevery = 10, signal_shape = (88200,), spect_shape = (662, 659)):
    song_id = 0
    ids = {}
    for song in songs:
        song_id += 1
        snippet_idx = 0
        # split the song into seconds intervals
        song_split = song.dice(1)
        # randomly sample SNIPPETS_PER_SONG snippets per song
        np.random.seed(song_id + snippet_idx)
        choices = np.random.choice(len(song_split),min(len(song_split),SNIPPETS_PER_SONG), replace = False)
        # iterate through the snippets
        for choice in choices:
            snippet = song_split[choice]
            snippet_idx += 1
            # create an identifier for the song/snippet combination
            snippet_name = "song_%d_snippet_%d"%(song_id,snippet_idx)
            # extract signal
            signal = snippet.to_numpy_array()
            if signal.shape != signal_shape:
                print ("Incorrect signal length on song %d. Skipping"%song_id)
                break
            # extract a spectrogram for the snippet
            _, _, spect = get_spect(snippet)
            if spect_shape != spect_shape:
                print ("Incorrect spectrogram shape on song %d. Skipping"%song_id)
                break
            # normalize to range (0,1)
            spect = normalize_spectogram(spect)
            if spect is None:
                continue
            # blur the spectrogram
            spect_blur = gaussian_blur(spect,window = (3,3))
            # edge detect
            spect_edges = edge_detect(spect,lowerbound = 0, upperbound=200,scale = 255)
            # perform max pooling to reduce dimensionality
            spect_blur = pooler(spect_blur)
            spect_edges = pooler(spect_edges)
            # create names for blurs, edges and signlas
            blur_name = snippet_name + "blur.npy"
            signal_name = snippet_name + "signal.npy"
            edges_name = snippet_name + "edges.npy"
            #add_names and save files
            ids[snippet_name] = {"blur":blur_name, "edges":edges_name, "signal":signal_name}
            save_npy(spect_blur,"../data/blurs/"+blur_name)
            save_npy(spect_edges,"../data/edges/"+edges_name)
            save_npy(signal,"../data/signals/"+signal_name)
        # print ever `print_every` songs
        if song_id % printevery ==0:
            print("Finished: %d" % song_id)
    # save the ids list
    with open("../data/ids_list.pkl","wb") as handle:
        pkl.dump(ids,handle)
    handle.close()

%time save_data(printevery=10)

Found: 3495 Files
Incorrect signal length on song 1. Skipping
Incorrect signal length on song 5. Skipping
Finished: 10
Incorrect signal length on song 19. Skipping
Incorrect signal length on song 20. Skipping
Finished: 20
Incorrect signal length on song 21. Skipping
Incorrect signal length on song 23. Skipping
Incorrect signal length on song 24. Skipping
Incorrect signal length on song 30. Skipping
Finished: 30
Incorrect signal length on song 32. Skipping
Incorrect signal length on song 33. Skipping
Finished: 40
Incorrect signal length on song 44. Skipping
Finished: 50
Finished: 60
Incorrect signal length on song 62. Skipping
Incorrect signal length on song 69. Skipping
Finished: 70
Incorrect signal length on song 72. Skipping
Finished: 80
Finished: 90
Incorrect signal length on song 95. Skipping
Finished: 100
Incorrect signal length on song 110. Skipping
Finished: 110
Incorrect signal length on song 118. Skipping
Finished: 120
Finished: 130
Incorrect signal length on song 135. Skippin

Incorrect signal length on song 1015. Skipping
Incorrect signal length on song 1017. Skipping
Incorrect signal length on song 1019. Skipping
Finished: 1020
Incorrect signal length on song 1023. Skipping
Finished: 1030
Incorrect signal length on song 1037. Skipping
Finished: 1040
Incorrect signal length on song 1050. Skipping
Finished: 1050
Incorrect signal length on song 1053. Skipping
Incorrect signal length on song 1054. Skipping
Incorrect signal length on song 1060. Skipping
Finished: 1060
Incorrect signal length on song 1067. Skipping
Incorrect signal length on song 1069. Skipping
Incorrect signal length on song 1070. Skipping
Finished: 1070
Incorrect signal length on song 1077. Skipping
Incorrect signal length on song 1080. Skipping
Finished: 1080
Incorrect signal length on song 1085. Skipping
Incorrect signal length on song 1087. Skipping
Finished: 1090
Incorrect signal length on song 1096. Skipping
Incorrect signal length on song 1099. Skipping
Finished: 1100
Finished: 1110
Inco

Finished: 1840
Finished: 1850
Incorrect signal length on song 1856. Skipping
Incorrect signal length on song 1858. Skipping
Incorrect signal length on song 1860. Skipping
Finished: 1860
Incorrect signal length on song 1865. Skipping
Incorrect signal length on song 1867. Skipping
Incorrect signal length on song 1868. Skipping
Incorrect signal length on song 1869. Skipping
Finished: 1870
Incorrect signal length on song 1871. Skipping
Incorrect signal length on song 1872. Skipping
Incorrect signal length on song 1873. Skipping
Incorrect signal length on song 1877. Skipping
Incorrect signal length on song 1878. Skipping
Finished: 1880
Incorrect signal length on song 1886. Skipping
Finished: 1890
Incorrect signal length on song 1891. Skipping
Incorrect signal length on song 1894. Skipping
Finished: 1900
Incorrect signal length on song 1902. Skipping
Incorrect signal length on song 1904. Skipping
Finished: 1910
Incorrect signal length on song 1911. Skipping
Incorrect signal length on song 19

Finished: 2650
Incorrect signal length on song 2652. Skipping
Incorrect signal length on song 2655. Skipping
Incorrect signal length on song 2659. Skipping
Incorrect signal length on song 2660. Skipping
Finished: 2660
Finished: 2670
Incorrect signal length on song 2671. Skipping
Finished: 2680
Incorrect signal length on song 2681. Skipping
Finished: 2690
Incorrect signal length on song 2692. Skipping
Finished: 2700
Incorrect signal length on song 2705. Skipping
Incorrect signal length on song 2707. Skipping
Finished: 2710
Incorrect signal length on song 2717. Skipping
Incorrect signal length on song 2718. Skipping
Incorrect signal length on song 2720. Skipping
Finished: 2720
Incorrect signal length on song 2729. Skipping
Finished: 2730
Finished: 2740
Incorrect signal length on song 2749. Skipping
Incorrect signal length on song 2750. Skipping
Finished: 2750
Incorrect signal length on song 2754. Skipping
Incorrect signal length on song 2757. Skipping
Incorrect signal length on song 2760

Incorrect signal length on song 3376. Skipping
Finished: 3380
Incorrect signal length on song 3381. Skipping
Incorrect signal length on song 3382. Skipping
Incorrect signal length on song 3383. Skipping
Incorrect signal length on song 3389. Skipping
Incorrect signal length on song 3390. Skipping
Finished: 3390
Finished: 3400
Incorrect signal length on song 3401. Skipping
Incorrect signal length on song 3402. Skipping
Incorrect signal length on song 3408. Skipping
Incorrect signal length on song 3410. Skipping
Finished: 3410
Incorrect signal length on song 3413. Skipping
Incorrect signal length on song 3414. Skipping
Incorrect signal length on song 3419. Skipping
Finished: 3420
Incorrect signal length on song 3423. Skipping
Finished: 3430
Finished: 3440
Incorrect signal length on song 3448. Skipping
Finished: 3450
Incorrect signal length on song 3452. Skipping
Incorrect signal length on song 3455. Skipping
Incorrect signal length on song 3456. Skipping
Incorrect signal length on song 34

In [35]:
np.random.choice(2,1,replace = False)

array([1])

In [36]:
min(1,2)

1

In [13]:
len(songs_split)

297